# Step 0. 환경 구성

구글 드라이브와 Colab을 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


전처리에 필요한 패키지 설치

In [ ]:
!pip install datasets>=2.6.1
!pip install transformers[torch]

# Step 1. 데이터 불러오기

압축 해제 및 음성,라벨링 데이터 리스트 형태로 저장

In [ ]:
import os
import shutil
import zipfile

def extract_zip_files(zip_file_path, extracted_folder_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder_path)

def change_folder_structure(extracted_folder_path):
    audio_target_dir = os.path.join(extracted_folder_path, "audio")
    label_target_dir = os.path.join(extracted_folder_path, "labels")

    # 폴더 생성
    os.makedirs(audio_target_dir, exist_ok=True)
    os.makedirs(label_target_dir, exist_ok=True)

    for root, _, files in os.walk(extracted_folder_path):
        for file in files:
            if file.endswith(".wav"):
                audio_file_path = os.path.join(root, file)
                new_audio_file_path = os.path.join(audio_target_dir, file)
                shutil.move(audio_file_path, new_audio_file_path)

            elif file.endswith(".json"):
                label_file_path = os.path.join(root, file)
                audio_id = file.split(".")[0]
                new_label_file_path = os.path.join(label_target_dir, f"{audio_id}.txt")
                shutil.move(label_file_path, new_label_file_path)

    # 빈 폴더 제거 (오직 audio_target_dir와 label_target_dir만 남아있을 것)
    for root, dirs, _ in os.walk(extracted_folder_path, topdown=False):
        for dir in dirs:
            if dir not in [os.path.basename(audio_target_dir), os.path.basename(label_target_dir)]:
                dir_path = os.path.join(root, dir)
                os.rmdir(dir_path)

# 1. 데이터 파일이 저장된 경로를 지정합니다.
zip_file_path_audio = '/content/drive/MyDrive/data_file/sample_final/audio_sample.zip'
zip_file_path_label = '/content/drive/MyDrive/data_file/sample_final/label_sample.zip'
extracted_folder_path = '/content/data/'  # 압축 해제된 데이터가 저장될 폴더 경로

# 폴더 초기화
if os.path.exists(extracted_folder_path):
    shutil.rmtree(extracted_folder_path)

# 2. zip 파일을 압축 해제합니다.
extract_zip_files(zip_file_path_audio, extracted_folder_path)
extract_zip_files(zip_file_path_label, extracted_folder_path)

# 3. 폴더 구조 변경
change_folder_structure(extracted_folder_path)

# 4. 구조 확인
# 데이터 폴더 경로 설정
data_folder = '/content/data/'

# audio 폴더 내 파일 목록 확인
audio_files = os.listdir(os.path.join(data_folder, 'audio'))
print("Audio Files:")
print(audio_files)

# labels 폴더 내 파일 목록 확인
label_files = os.listdir(os.path.join(data_folder, 'labels'))
print("\nLabel Files:")
print(label_files)

# 이제 "/content/data/" 폴더 내에 "audio" 폴더와 "labels" 폴더가 생성되며,
# 해당 폴더에 원천데이터인 wav 파일과 라벨링데이터인 txt 파일이 저장됩니다.
# 이후에는 해당 경로에서 데이터를 불러와서 학습을 진행하시면 됩니다.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



audio data 개수와 label data 개수 같은지 확인

In [ ]:
import os

data_folder = "/content/data"
audio_folder = os.path.join(data_folder, "audio")
label_folder = os.path.join(data_folder, "labels")

# audio 폴더 내의 파일 개수 확인
audio_files = [file for file in os.listdir(audio_folder) if file.endswith(".wav")]
print(f"오디오 파일 개수: {len(audio_files)}")

# labels 폴더 내의 파일 개수 확인
label_files = [file for file in os.listdir(label_folder) if file.endswith(".txt")]
print(f"JSON 파일 개수: {len(label_files)}")

오디오 파일 개수: 65503
JSON 파일 개수: 65503


# Step2. 데이터셋 구조 변경

train과 test 데이터로 나누기 및 모델 학습을 위한 구조 변경

In [ ]:
from datasets import DatasetDict, Dataset, load_dataset
import os
import json

def create_dataset(data_folder):
    audio_folder = os.path.join(data_folder, "audio")
    label_folder = os.path.join(data_folder, "labels")

    # audio 폴더 내의 WAV 파일 리스트를 가져옴
    audio_files = [file for file in os.listdir(audio_folder) if file.endswith(".wav")]

    # 데이터셋 생성
    dataset_dict = DatasetDict()
    dataset = {"audio": [], "sentence": []}

    # 라벨 파일의 "FileName"을 기준으로 오디오 파일 찾아서 연결
    for label_file in os.listdir(label_folder):
        label_path = os.path.join(label_folder, label_file)

        with open(label_path, "r", encoding="utf-8") as f:
            label_data = json.load(f)
            audio_filename = label_data["File"]["FileName"]
            sentence = label_data["Transcription"]["LabelText"]

            # "FileName"과 일치하는 오디오 파일 찾아서 연결
            matching_audio = [audio_file for audio_file in audio_files if audio_filename in audio_file]
            if matching_audio:
                audio_path = os.path.join(audio_folder, matching_audio[0])
                dataset["audio"].append(audio_path)
                dataset["sentence"].append(sentence)

    # 데이터량 36000개 이상이 되면 시스템 Ram 51GB 넘어감
    train_size = 25000
    test_size = 1000

    # 고정된 인덱스 리스트 생성
    data_size = len(dataset["audio"])
    indices = list(range(data_size))

    train_data = {"audio": [], "sentence": []}
    test_data = {"audio": [], "sentence": []}

    # Train 데이터 추출
    for i in indices[train_size+test_size:train_size+train_size+test_size]:
        train_data["audio"].append(dataset["audio"][i])
        train_data["sentence"].append(dataset["sentence"][i])

    # Test 데이터 추출
    for i in indices[train_size+train_size+test_size:train_size+test_size+train_size+test_size]:
        test_data["audio"].append(dataset["audio"][i])
        test_data["sentence"].append(dataset["sentence"][i])

    # 데이터셋에 추가
    dataset_dict["train"] = Dataset.from_dict(train_data)
    dataset_dict["test"] = Dataset.from_dict(test_data)

    return dataset_dict

# 데이터 폴더 경로 지정
data_folder = "/content/data"

# 데이터셋 생성
custom_dataset = create_dataset(data_folder)

# "sentence" 필드만 출력
print(custom_dataset)


DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1000
    })
})


# Step 3. 데이터 전처리

prepare_dataset 함수에서 필요한 feature_extractor, tokenizer 구하기

In [ ]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer

# - Load Feature extractor: WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

# - Load Tokenizer: WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="korean", task="transcribe")

원본 데이터를 whisper 모델에 입력이 가능한 형식으로 변환하여 학습이 가능하도록 하는 작업

In [ ]:
def prepare_dataset(batch):
    """
    Prepare audio data to be suitable for Whisper AI model.
    """
    # 오디오 데이터를 48kHz에서 16kHz로 변환하여 로드
    audio = batch["audio"]

    # 입력 오디오 배열로부터 log-Mel 입력 특성을 계산
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # 대상 텍스트를 토큰화하고 라벨 ID로 인코딩
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

print('| Check the audio example')
print(f'{custom_dataset["train"][0]}\n')

# Downsample from 48kHZ to 16kHZ
# Whisper AI 모델이 16kHz의 샘플링 속도에서 훈련되어서
# 입력 데이터를 16kHz로 다운샘플링하는 것이 필요
from datasets import Audio
custom_dataset = custom_dataset.cast_column("audio", Audio(sampling_rate=16000))

print('| Check the effect of downsampling:')
print(f'{custom_dataset["train"][0]}\n')

# Prepare and use function to prepare our data ready for the Whisper AI model
custom_dataset = custom_dataset.map(
    prepare_dataset,
    remove_columns=custom_dataset.column_names["train"],
    num_proc=2 # num_proc > 1 will enable multiprocessing
    )

# Step 4. 전처리한 데이터 저장 및 불러오기

map 함수를 적용시킨 데이터셋을 구글 드라이브에 저장

In [ ]:
# Save the custom_dataset to the specified Google Drive path
custom_dataset.save_to_disk('/content/drive/MyDrive/data_file/map_data_25000_ver2')

map 적용시킨 데이터셋을 불러오기

In [ ]:
from datasets import load_from_disk
loaded_custom_dataset1 = load_from_disk('/content/drive/MyDrive/data_file/map_data_25000_ver1')
loaded_custom_dataset2 = load_from_disk('/content/drive/MyDrive/data_file/map_data_25000_ver2')

데이터셋 구조 확인

In [ ]:
print(loaded_custom_dataset1)
print(loaded_custom_dataset2)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1000
    })
})
DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1000
    })
})


# Step 5. 데이터셋 결합

데이터셋 결합

In [ ]:
from datasets import concatenate_datasets

# Concatenate 'train' split of custom datasets
combined_train_dataset = concatenate_datasets([loaded_custom_dataset1['train'], loaded_custom_dataset2['train']])

# Concatenate 'test' split of custom datasets
combined_test_dataset = concatenate_datasets([loaded_custom_dataset1['test'], loaded_custom_dataset2['test']])

# Save the combined 'train' dataset
combined_train_dataset.save_to_disk('/content/drive/MyDrive/data_file/combined_train_dataset')

# Save the combined 'test' dataset
combined_test_dataset.save_to_disk('/content/drive/MyDrive/data_file/combined_test_dataset')


결합된 각각의 train 데이터와 test 데이터를 불러오기

In [ ]:
from datasets import load_from_disk

# Load the combined 'train' dataset
loaded_combined_train_dataset = load_from_disk('/content/drive/MyDrive/data_file/combined_train_dataset')

# Load the combined 'test' dataset
loaded_combined_test_dataset = load_from_disk('/content/drive/MyDrive/data_file/combined_test_dataset')

# Now you can use loaded_combined_train_dataset and loaded_combined_test_dataset as needed
# For example, you can access specific examples using indexing like loaded_combined_train_dataset[0]


In [ ]:
print(loaded_combined_train_dataset)
print(loaded_combined_test_dataset)

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 50000
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2000
})


코렙의 제한된 시스템 Ram 용량 및 제한된 리소스 사용시간 때문에 데이터셋을 나누어 전처리 과정을 진행

전처리 과정이 끝난후에는 다시 결합하여 저장한 다음 학습시킬때 전처리한 데이터셋을 불러와서 학습을 진행할수 있도록 함